<a href="https://colab.research.google.com/github/alyagabsi/inventory-prediction/blob/main/Order_Optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Ordering Optimization Problem**
_______________

Please see the formulation of the problem [here](https://drive.google.com/file/d/1XMsGGo5HXC9JMXMm8wkVrzbxwz5PSyob/view?usp=sharing).

## **Setting Dependencies**

In [1]:
# Importing packages 

!pip install pulp
import pandas as pd 
import numpy as np
from pulp import * 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.2 MB 19.8 MB/s 


In [2]:
# data 

products = ['Apples', 'Beans', 'Beets', 'Broccoli', 'Cabbage',
            'Carrots', 'Celery', 'Chickpeas', 'Cucumber',
            'Eggs', 'Garlic', 'Ginger', 'Lentils',
            'Milk', 'Onions', 'Oranges','Potatoes',
            'Rice', 'Salad', 'Spinach', 'Sprouts', 'Squash']

units = ['lbs', 'kg', 'lbs','unit', 'lbs', 'lbs', 'unit', 'kg',
         'unit', 'dozen', 'lbs', 'lbs', 'kg', 'litre',
         'lbs', 'unit', 'lbs', 'kg', 'kg', 'unit','kg', 'lbs']

units_dict = {}

for i in range(len(products)):
  units_dict[products[i]] = units[i] 


In [ ]:
# User input 

C = {} # quote per unit
I = {} # supplier inventory
G = {} # produced in garden
CL = {} # carry over leftovers
L = {} # allowable leftovers

for product in products:
  print('Please input the following information for ' + product + ':')
  print('')
  C[product] = float(input('Supplier quote per ' + units[product]+': '))
  print('')
  I[product] = float(input('Supplier inventory in ' + units[product]+ ': '))
  print('')
  G[product] = float(input('Supply of grown ' + product + ' in ' + units[product]+ ': '))
  print('')
  CL[product] = float(input('Carry over leftovers in '+ units[product] + ': '))
  print('')
  L[product] = float(input('Allowable leftovers in '+ units[product] + ': '))
  print('')

# Demand (given by forecasting models) - TBDone

D = {}

for product in products:
  D[product] = forecast[i] # -> forecast list to be implemented 

# Budget 

B = float(input('Please input budget:'))

## **Formulating the Problem**

In [3]:
# Initializing model

model = LpProblem('Order_Optimization', LpMinimize)

Setting Decision Variables 

In [4]:
# Amount to order from supplier 

X = LpVariable.dicts('Order', products, lowBound = 0, cat = 'Continuous')

#  Sales 

S = LpVariable.dicts('Sales', products, lowBound =0, cat='Continuous')

Setting Objective Function

In [5]:
model += sum([X[product]*C[product] for product in products])

NameError: ignored

Setting Constraints

*Might need to impose a new variable to not satisfy the demand fully*

In [ ]:
for product in products:
  model += X[product] + G[product] + CL[product] >= D[product] # Demand constraints
  model += -X[product] >= -I[product] # Dupplier constraints
  model += -S[product] >= -D[product] # Sales constraints
  model += -(X[product] + G[product] + CL[product] - S[product]) >= -L[product] # Leftover constraints

In [ ]:
# budget constraints 

model += sum([X[product]*C[product] for product in products]) <= B

In [ ]:
print(model)

Order_Optimization:
MINIMIZE
10.0*Order_Apples + 10.0*Order_Oranges + 10.0*Order_Potatoes + 0.0
SUBJECT TO
_C1: Order_Apples >= -10

_C2: - Order_Apples >= -10

_C3: - Sales_Apples >= -10

_C4: - Order_Apples + Sales_Apples >= 10

_C5: Order_Oranges >= -10

_C6: - Order_Oranges >= -10

_C7: - Sales_Oranges >= -10

_C8: - Order_Oranges + Sales_Oranges >= 10

_C9: Order_Potatoes >= -10

_C10: - Order_Potatoes >= -10

_C11: - Sales_Potatoes >= -10

_C12: - Order_Potatoes + Sales_Potatoes >= 10

VARIABLES
Order_Apples Continuous
Order_Oranges Continuous
Order_Potatoes Continuous
Sales_Apples Continuous
Sales_Oranges Continuous
Sales_Potatoes Continuous



## **Solving LP**

In [ ]:
result = model.solve()

Printing Results



In [ ]:
print('Optimial Values:')
print('')
for var in model.variables():
  print(var.name, '==>', var.varValue)

print('')
print('Expected Total cost: $',pulp.value(model.objective))